In [1]:
# Import all required libraries
import random

from mingus.containers import Note
from mingus.midi import midi_file_out
from mingus.containers import Track, Bar

import pygame

import time

pygame 2.6.0 (SDL 2.28.4, Python 3.9.19)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
# Create example note sequence -> C Major Scale
notes = ['C', 'D', 'E', 'F', 'G', 'A', 'B']

In [3]:
# Define transition matrix for the notes
transition_matrix = {
    'C': ['D', 'E', 'F', 'G', 'A', 'B'],
    'D': ['C', 'E', 'F', 'G', 'A', 'B'],
    'E': ['C', 'D', 'F', 'G', 'A', 'B'],
    'F': ['C', 'D', 'E', 'G', 'A', 'B'],
    'G': ['C', 'D', 'E', 'F', 'A', 'B'],
    'A': ['C', 'D', 'E', 'F', 'G', 'B'],
    'B': ['C', 'D', 'E', 'F', 'G', 'A'],
}

In [4]:
# Generate music function
def generate_music(start_note, length=20, octave=4):
    music = []
    current_note = start_note

    for _ in range(length):
        next_note = random.choice(transition_matrix[current_note])
        next_note_obj = Note(next_note, octave)
        music.append(next_note_obj)
        current_note = next_note

    return music

In [5]:
# Generate a music sequence starting from 'C' in the n-th octave with x-length of notes
generated_music = generate_music('C', length=10, octave=3)
print("Generated music sequence:", [str(note) for note in generated_music])

Generated music sequence: ["'G-3'", "'C-3'", "'F-3'", "'B-3'", "'G-3'", "'B-3'", "'G-3'", "'C-3'", "'B-3'", "'A-3'"]


In [6]:
# Create a Track and add the sequence as Bars with notes
track = Track()
bar = Bar()

In [7]:
# Add notes to the bar with a fixed duration (e.g., 4 beats per note)
for note in generated_music:
    if bar.is_full():
        track.add_bar(bar)
        bar = Bar()  # Start a new bar if the current one is full
    bar.place_notes(note, 4)  # 4 beats per note

In [8]:
# Add the last bar even if it's not full
track.add_bar(bar)

# Write the Track to a MIDI file
midi_filename = "simple_melody.mid"
midi_file_out.write_Track(midi_filename, track)

True

In [9]:
# Initialize Pygame and play the MIDI file
pygame.init()
pygame.mixer.init()
pygame.mixer.music.load(midi_filename)
pygame.mixer.music.play()

# Wait until the music finishes playing
while pygame.mixer.music.get_busy():
    time.sleep(1)

pygame.quit()